In [1]:
import requests
from bs4 import BeautifulSoup
import regex as re

In [2]:
def access_link(link):
    return requests.get(link,headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.2 Safari/605.1.15'})

response = access_link('https://haveyoursay.tfl.gov.uk/projects')

soup = BeautifulSoup(response.text,'html.parser')


In [3]:
links = soup.find_all('a', class_='project-tile__link')
for tag in links:
    print(tag['href'])

/cranford-parkway-junction
/a1-archway-road
/a10-lincoln-road-junction
/merantun-way
/310-holloway-nags-head
/meridian-water-bus
/dlr-extension-consultation-2
/routes-259-279-349
/bus-routes-19-38
/barnham-drive
/oxford-street-transport-highways
/thamesmead-proposals
/bus-424-putney-fulham
/a232-healthy-streets
/taxi-fares-review-2025
/sl15-superloop
/sl14-superloop
/earls-court-road-works
/barnes-bus-services
/sl12-superloop
/sl13-superloop
/pedicabs-licensing
/lsa
/carshalton-high-street-bus-priority-improvements
/routes-72-283
/shoreditch-high-street
/lowering-speed-limits
/capital-call
/youth-panel-application
/bus-routes-287-687
/673-bus-route
/congestion-charge-proposals
/routes-n118-114-proposals
/route-698
/bilton-way
/605-bus-route
/a21-hastings-road
/superloop-expansion
/sl11-superloop
/oxford-street
/cycleway-c62
/bl1-superloop
/pedicab-regulations
/654-bus-route
/east-putney-improvements
/319-weekend-night-service
/shoreditch
/bus-routes-346-347-497
/14-414-proposals
/cross

In [4]:
from urllib.parse import urljoin
project_links = {}
for project_link in links:
    link = urljoin('https://haveyoursay.tfl.gov.uk',project_link['href'])
    project_name = str(project_link['href'])[1:]
    project_links[project_name] = link

print(project_links)

{'cranford-parkway-junction': 'https://haveyoursay.tfl.gov.uk/cranford-parkway-junction', 'a1-archway-road': 'https://haveyoursay.tfl.gov.uk/a1-archway-road', 'a10-lincoln-road-junction': 'https://haveyoursay.tfl.gov.uk/a10-lincoln-road-junction', 'merantun-way': 'https://haveyoursay.tfl.gov.uk/merantun-way', '310-holloway-nags-head': 'https://haveyoursay.tfl.gov.uk/310-holloway-nags-head', 'meridian-water-bus': 'https://haveyoursay.tfl.gov.uk/meridian-water-bus', 'dlr-extension-consultation-2': 'https://haveyoursay.tfl.gov.uk/dlr-extension-consultation-2', 'routes-259-279-349': 'https://haveyoursay.tfl.gov.uk/routes-259-279-349', 'bus-routes-19-38': 'https://haveyoursay.tfl.gov.uk/bus-routes-19-38', 'barnham-drive': 'https://haveyoursay.tfl.gov.uk/barnham-drive', 'oxford-street-transport-highways': 'https://haveyoursay.tfl.gov.uk/oxford-street-transport-highways', 'thamesmead-proposals': 'https://haveyoursay.tfl.gov.uk/thamesmead-proposals', 'bus-424-putney-fulham': 'https://haveyours

In [8]:
import time
import json
import mimetypes

try:
    with open('data/web_data.json', 'r') as f:
        project_data = json.load(f)
    print('data/web_data.json found, skipping web scrape...')
except FileNotFoundError:
    project_data = {}

    items = list(project_links.items())

    for i, item in enumerate(items):
        
        pname = item[0]
        plink = item[1]
        print('Beginning scrape of ' + pname)

        if pname in project_data:
            print(pname + ' found, skipping...')
            continue
        else:
            response = access_link(plink)
            soup = BeautifulSoup(response.text,'html.parser')    

            project_texts = soup.find_all(class_=re.compile('project_widgets')) + soup.find_all(id=re.compile('project_description_text'))
            ptext = ''
            for txt in project_texts:
                ptext += txt.get_text(separator='\n')
            
            project_data[pname] = {}
            project_data[pname]['project_text'] = ptext
            
            documents = soup.find_all('a',class_=re.compile('document'))
            file_number = 0
            project_data[pname]['documents'] = []

            for j, file in enumerate(documents):
                download_link = file['href'] + '/download'

                pdf_response = access_link(download_link)
                print(pdf_response.headers.get('Content-Type'))
                ext = mimetypes.guess_extension(pdf_response.headers.get('Content-Type').split(';')[0])
                if ext == None:
                    print('file extension not found, aborting')
                    break
                document_name = pname + '_' + str(j) + ext
                filepath = 'documents/' + document_name
                with open(filepath,'wb') as f:
                    f.write(pdf_response.content)
                print('Saving file: ' + document_name)
                project_data[pname]['documents'].append(filepath)
            
            time.sleep(1)
        
        with open('web_data.json','w') as f:
            json.dump(project_data,f)
            print(f'Web-scraped data saved under {f}')

    



data/web_data.json found, skipping web scrape...
